# Imports

In [1]:
!pip install pytube --quiet

     |████████████████████████████████| 51kB 3.8MB/s 


In [2]:
from bs4 import BeautifulSoup
import requests
import json
import re
import urllib.request
from pytube import YouTube

# Helper Functions

In [3]:
class Helper:
    def __init__(self):
        pass

    def title_to_underscore_title(self, title: str):
        title = re.sub("[\W_]+", "_", title)
        return title.lower()

    def id_from_url(self, url: str):
        return url.rsplit('=', 1)[1]


class YouTubeStats:
    def __init__(self, url: str):
        #self.json_url = urllib.request.urlopen(url)
        self.json_url = requests.get(url)
        self.data = json.loads(self.json_url.text)

    def print_data(self):
        print(self.data)

    def get_video_title(self):
        # pass
        return self.data["items"][0]["snippet"]["title"]

    def get_video_description(self):
        # pass
        return self.data["items"][0]["snippet"]["description"]

    def download_video(self, youtube_url: str, title: str):
        YouTube(youtube_url).streams.first().download(PATH)

# Playlist and API info

In [4]:
#Path to DL the videos
PATH = 'insert path to DL files to'

playlist = 'insert playlist'
api_key = 'insert youtube API key'

# Link scraping

In [3]:
# Scraping links
import googleapiclient.discovery
from urllib.parse import parse_qs, urlparse

#extract playlist id from url
query = parse_qs(urlparse(playlist).query, keep_blank_values=True)
playlist_id = query["list"][0]

# print(f'get all playlist items links from {playlist_id}')
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey = api_key)

request = youtube.playlistItems().list(
    part = "snippet",
    playlistId = playlist_id,
    maxResults = 50
)
response = request.execute()

playlist_items = []
while request is not None:
    response = request.execute()
    playlist_items += response["items"]
    request = youtube.playlistItems().list_next(request, response)

# print(f"total: {len(playlist_items)}")
links = [f'/watch?v={t["snippet"]["resourceId"]["videoId"]}' for t in playlist_items]

with open("FILE PATH TO SAVE CSV OF LINKS", "w") as outfile:
    outfile.write("\n".join(str(item) for item in links))

len(links)

#Manual adds for various playlist csv's

Add to Feb_21 csv_list ***/watch?v=KzC4UPH9A5A***

# Scrape the videos

In [1]:
# This linked file contains the links... could make a seperate list within this script instead. 
link_file = "FILE PATH TO OPEN CSV OF LINKS"
with open(link_file, "r") as f:
    content = f.readlines()

content = list(map(lambda s: s.strip(), content))
content = list(map(lambda s: s.strip(','), content))

helper = Helper()

for youtube_url in content[:]:
    video_id = helper.id_from_url(youtube_url)
    url = f"https://www.googleapis.com/youtube/v3/videos?part=snippet&id={video_id}&key={api_key}"
    # print(url)
    yt_stats = YouTubeStats(url)
    title = yt_stats.get_video_title()
    title = helper.title_to_underscore_title(title)
    print(title)
    # description = yt_stats.get_video_description()
    # print(description)
    yt_stats.download_video(youtube_url, title)


# Scrape one video

In [2]:
# One at a time

#video_id = the 10 characters after  '/watch?v='
video_id = 'KzC4UPH9A5A'
url = f"https://www.googleapis.com/youtube/v3/videos?part=snippet&id={video_id}&key={api_key}"
yt_stats = YouTubeStats(url)
title = yt_stats.get_video_title()
helper = Helper()
title = helper.title_to_underscore_title(title)
print(title)
# description = yt_stats.get_video_description()
# print(description)
yt_stats.download_video('/watch?v=KzC4UPH9A5A', title)